<a href="https://colab.research.google.com/github/RadouaneElarfaoui/Animax/blob/main/Animax_fb_v1_0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Upload anime to Facebook by API**

## **Update V 1.0.2:**
- Upload any Anime from [witanime](https://witanime.cyou/)
- Two sources of download:
  - Mediafire.
  - Google Drive.
- Download list of episodes (from episode X to episode Y).
- Download episodes in Full HD quality.
- Save logs in Google Sheets table.
- Personalize post titles and descriptions.

---

**If you don't have a Google Cloud service account, choose `without save`.**

# **Requirements:**
- Create an app in [Facebook for Developers](https://developers.facebook.com/).
- Get page access token with permissions `publish_video`, `read engagement pages`, and `page manage posts`.
- Get page ID.
- Get any Anime episode URL from [witanime](https://witanime.cyou/), and remove the episode number from the URL.

---

| Original URL | New URL |
|--------------|---------|
| https://witanime.cyou/episode/kimetsu-no-yaiba-hashira-geiko-hen-%d8%a7%d9%84%d8%ad%d9%84%d9%82%d8%a9-1/ | https://witanime.cyou/episode/kimetsu-no-yaiba-hashira-geiko-hen-%d8%a7%d9%84%d8%ad%d9%84%d9%82%d8%a9- |

---

* Run the two functions first.
* Complete the necessary information in the form.
* Run program ▶

# **Thanks to:**
- [Radouane Elarfaoui](https://web.facebook.com/A.R.F.RADOUANE)
- [Google Colab](https://colab.research.google.com/)
- [Google AI Studio](https://aistudio.google.com/)

**© 2024 [Alfred Pro](https://web.facebook.com/pro.alfred.fb/). All rights reserved.**

In [ ]:
 # @title download_ep(url, ep)

# This is a sample Python script.

# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

import requests
from bs4 import BeautifulSoup
import re
import base64
import os
import zipfile
from tqdm import tqdm

def decode_link(encoded_link):
    try:
        decoded_bytes = base64.b64decode(encoded_link)
        decoded_link = decoded_bytes.decode('utf-8')
        return decoded_link
    except Exception as e:
        print(f"Failed to decode link: {encoded_link}, Error: {e}")
        return None

def get_download_link(url):
    try:
        response = requests.get(url, allow_redirects=True)
        if response.status_code != 200:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return {}

        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouver toutes les balises <a> avec la classe 'btn btn-default download-link' dans <ul class="quality-list">
        ul_element = soup.find('ul', class_='quality-list')
        if ul_element is None:
            print("No 'ul' element with class 'quality-list' found.")
            return {}

        download_links = ul_element.find_all('a', class_='btn btn-default download-link')
        source_list = [source.find('span', class_='notice').text for source in download_links]

        # Trouver le script avec l'ID spécifié
        script = soup.find('script', {'id': 'theme-dlls-js-extra'})

        if script:
            script_content = script.string
            if script_content:
                # Utiliser une expression régulière pour extraire downloadUrls
                pattern = r'var downloadUrls = (\{.*?\});'
                match = re.search(pattern, script_content, re.DOTALL)
                if match:
                    download_urls_json = match.group(1)
                    download_urls_dict = eval(download_urls_json)  # Convertir la chaîne JSON en dictionnaire

                    # Combiner source_list et download_urls_dict en un dictionnaire
                    result_dict = {}
                    for source, (key, encoded_link) in zip(source_list, download_urls_dict.items()):
                        decoded_link = decode_link(encoded_link)
                        if decoded_link:
                            result_dict[source] = decoded_link

                    return result_dict

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return {}

    return {}


def download_and_process_mediafire_file(url, destination_folder, ep):
    try:
        # Obtenir le contenu de la page Mediafire
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve the Mediafire page. Status code: {response.status_code}")
            return None

        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouver le lien de téléchargement direct
        download_link = soup.find('a', {'id': 'downloadButton'})
        if download_link is None:
            print("No download button found.")
            return None

        download_url = download_link['href']
        print(f"Downloading from {download_url}")

        # Télécharger le fichier avec une barre de progression
        download_response = requests.get(download_url, stream=True)
        if download_response.status_code != 200:
            print(f"Failed to download the file. Status code: {download_response.status_code}")
            return None

        # Extraire le nom du fichier à partir de l'URL
        file_name = download_url.split('/')[-1]
        if file_name.endswith('.mp4'):
            file_name = "EP_"+str(ep)+".mp4"
        elif file_name.endswith('.zip'):
            file_name = "EP_"+str(ep)+".zip"

        print(f"File name: {file_name}")


        file_path = os.path.join(destination_folder, file_name)

        # Obtenir la taille totale du fichier pour la barre de progression
        total_size = int(download_response.headers.get('content-length', 0))

        # Sauvegarder le fichier téléchargé avec une barre de progression
        with open(file_path, 'wb') as f, tqdm(
            desc=file_name,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in download_response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    bar.update(len(chunk))
        print(f"File downloaded successfully and saved to {file_path}")

        # Vérifier si le fichier est un zip ou non
        if not file_path.endswith('.mp4') and zipfile.is_zipfile(file_path):
            # Décompresser le fichier
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(destination_folder)
                extracted_files = zip_ref.namelist()

            # Renommer le fichier décompressé
            extracted_file_path = os.path.join(destination_folder, extracted_files[0])
            new_file_path = os.path.join(destination_folder, file_name.replace('.zip', '.mp4'))
            os.rename(extracted_file_path, new_file_path)

            # Supprimer le fichier zip original
            os.remove(file_path)

            return new_file_path

        return file_path

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


def download_file_from_google_drive(url, destination_folder , ep):
    try:
        file_name = "EP_"+str(ep)+".mp4"
        # Extract file ID from URL
        file_id = url.split('?id=')[1].split('&')[0]
        download_url = f"https://drive.google.com/uc?id={file_id}&export=download"

        # Define destination path
        output = os.path.join(destination_folder, file_name)

        # Download file with progress bar
        download_response = session.get(download_url, stream=True)
        if download_response.status_code != 200:
            print(f"Failed to download the file from Google Drive. Status code: {download_response.status_code}")
            return None

        total_size = int(download_response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='B', unit_scale=True, desc=file_name)

        with open(output, 'wb') as f:
            for data in download_response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()

        print(f"File downloaded successfully and saved to: {output}")

        # Check if downloaded file is a ZIP file
        if not output.endswith('.mp4') and zipfile.is_zipfile(output):
            # Extract the ZIP file
            with zipfile.ZipFile(output, 'r') as zip_ref:
                zip_ref.extractall(destination_folder)
                extracted_files = zip_ref.namelist()

            # Rename the extracted file
            extracted_file_path = os.path.join(destination_folder, extracted_files[0])
            new_file_path = os.path.join(destination_folder, file_name.replace('.zip', ''))
            os.rename(extracted_file_path, new_file_path)

            # Remove the original ZIP file
            os.remove(output)

            return new_file_path

        return output

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during download: {e}")
        return None


def download_ep(url, ep):
    destination_folder = './downloads'
    # Créer le répertoire
    os.makedirs(destination_folder, exist_ok=True)

    episode_url = f"{url}{ep}/"
    print(f"Url of episode {ep} in witanime: {episode_url}")
    link_list = get_download_link(episode_url)
    if not link_list:
        print("No download links found.")
        return
    if 'mediafire' in link_list:
            mediafire_link = link_list['mediafire']
            print(f"Mediafire link: {mediafire_link}")
            # Usage
            file_path = download_and_process_mediafire_file(mediafire_link, destination_folder, ep)
            if file_path:
                print(f"File downloaded and processed successfully: {file_path}")
    elif 'google drive' in link_list:
        google_drive_link = link_list['google drive']
        print(f"Google Drive link: {google_drive_link}")
        # Usage
        file_path = download_file_from_google_drive(google_drive_link, destination_folder ,ep)
        if file_path:
            print(f"File downloaded and processed successfully: {file_path}")
    else:
        return
    return file_path




if __name__ == '__main__':
    # Exemple d'utilisation
    #ep = 372
    #url = "https://witanime.quest/episode/one-piece-%D8%A7%D9%84%D8%AD%D9%84%D9%82%D8%A9-"
    #download_ep(url, ep)
    pass


In [ ]:
# @title upload_ep_2_fb(file,ep_nm,video_title,video_description,page_id,access_token)

# my script=====================

import requests
def upload_ep_2_fb(file,video_title,video_description,page_id,access_token):
    # L'URL de l'API pour le téléchargement de vidéos
    url = f'https://graph-video.facebook.com/v20.0/{page_id}/videos'

    # Préparez les paramètres de la requête
    files = {
        'file': (file, open(file, 'rb')),
        'title': (None, video_title),
        'description': (None, video_description)
    }

    # Préparez les en-têtes de la requête
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    # Effectuez la requête POST pour télécharger la vidéo
    response = requests.post(url, headers=headers, files=files)

    # Vérifiez la réponse
    print('Réponse :', response.text)
    if response.ok:
        print('Vidéo publiée avec succès !', video_title)
        print('ID de la vidéo :', response.json()['id'])
        return True, response.text , response.json()['id']
    else:
        print('Échec de la publication de la vidéo.')
        return False , response.text , None

In [ ]:
# @title #**Download and Upload Episodes to My Facebook Page**
# Set up variables
anime_name = "One Piece"  # @param {type:"string"}
page_id = "338579942662683"  # @param {type:"string"}
access_token = "EAAOEIWrK8f7yXqgvyO6ZAS1a6pVffedYvV83ldG7NmDt5cvVpasifvYEg45CUPZAZByyfmVz6Cyg0q5o40r0Ch4l6DPLpwH50zp3fjhfcSWBpounYSvGsIRAT89DfLgg7RD1BqEbSkyJEE6YB0mhJl"  # @param {type:"string"}
start_ep = 37  # @param {type:"integer"}
end_ep = 61  # @param {type:"integer"}
url = "https://witanime.quest/episode/one-piece-%D8%A7%D9%84%D8%AD%D9%84%D9%82%D8%A9-"  # @param {type:"string"}
video_title_lin1 = "وان بيس | الحلقة #EP"  # @param {type:"string"}
video_title_lin2 = ""  # @param {type:"string"}
video_description_line1 = "وان بيس | الحلقة #EP"  # @param {type:"string"}
video_description_line2 = "أصدقائي، قدروا تعبنا في نشر هذه الحلقة. ادعمونا بالضغط على زر الإعجاب وترك تعليق محفز، ولا تنسوا متابعة الصفحة للمزيد من الحلقات الممتعة!"  # @param {type:"string"}
save_method = "without save"  # @param ["without save", "google sheet"]
folder_id = "1E4uRKRlNCTJU9R6GQIXMfZqpl1Y9ztPs"  # @param {type:"string"}

# @markdown ---
# @markdown **Save Method:**
# @markdown
# @markdown - If you choose the **Google Sheet** save method:
# @markdown   - You should enter the ID of any Google Drive folder to save logs. If left empty, the logs will be saved in the parent folder.
# @markdown - If you have chosen **without save**, leave this field empty.
# @markdown ---


#################################################
################# Start Program #################
#################################################

def download_and_upload_episodes_with_save(start_ep, end_ep, url, page_id, access_token, folder_id):
    """
    Downloads and uploads episodes to Facebook with Google Sheets save method.
    """
    try:
        from google.colab import auth
        auth.authenticate_user()
        import gspread
        from google.auth import default

        spreadsheet_name = f"{anime_name} EP_[{start_ep} - {end_ep}]"
        creds, _ = default()
        gc = gspread.authorize(creds)

        # Check if spreadsheet exists
        spreadsheet_exists = False
        for file in gc.list_spreadsheet_files():
            if file['name'] == spreadsheet_name:
                spreadsheet_exists = True
                break

        if not spreadsheet_exists:
            sh = gc.create(spreadsheet_name, folder_id=folder_id)
        else:
            sh = gc.open(spreadsheet_name)

        print(f"Spreadsheet created/loaded: {sh.title}")

        # Clear and update header in worksheet
        wh = sh.worksheets()[0]
        wh.clear()
        wh.update_title(spreadsheet_name)
        wh.resize(cols=8)

        row = ["Anime Name", "Page ID", "Episode", "Video Title", "Video Description", "Status", "Video ID", "Response"]
        cell_list = wh.range('A1:H1')
        for i, val in enumerate(row):
            cell_list[i].value = val
        wh.update_cells(cell_list)

        # Download and upload episodes
        for ep in range(start_ep, end_ep + 1):
            file_path = download_ep(url, ep)
            if file_path:
                video_title_lin1_updated = video_title_lin1.replace("#EP", str(ep))
                video_title_lin2_updated = "\n" + video_title_lin2 if video_title_lin2 else ""
                video_title = video_title_lin1_updated + video_title_lin2_updated

                video_description_line1_updated = video_description_line1.replace("#EP", str(ep))
                video_description_line2_updated = "\n" + video_description_line2 if video_description_line2 else ""
                video_description = video_description_line1_updated + video_description_line2_updated

                print(f"Uploading episode {ep} to Facebook...")
                print(f"Title: {video_title}")
                print(f"Description: {video_description}")
                print(f"File path: {file_path}")

                upload_successful, response, video_id = upload_ep_2_fb(file_path, video_title, video_description, page_id, access_token)

                row = [anime_name, page_id, ep, video_title, video_description, upload_successful, video_id, response]
                wh.append_row(row)

                if upload_successful:
                    print(f"Episode {ep} uploaded successfully to Facebook.")
                else:
                    print(f"Failed to upload episode {ep} to Facebook.")
                    break
            else:
                print(f"Failed to download episode {ep}.")
                row = [anime_name, page_id, ep, "_", "_", "Failed", "_", "_"]
                wh.append_row(row)

    except Exception as e:
        print(f"An error occurred: {e}")

def download_and_upload_episodes_without_save(start_ep, end_ep, url, page_id, access_token):
    """
    Downloads and uploads episodes to Facebook without saving to Google Sheets.
    """
    try:
        for ep in range(start_ep, end_ep + 1):
            file_path = download_ep(url, ep)
            if file_path:
                video_title_lin1_updated = video_title_lin1.replace("#EP", str(ep))
                video_title_lin2_updated = "\n" + video_title_lin2 if video_title_lin2 else ""
                video_title = video_title_lin1_updated + video_title_lin2_updated

                video_description_line1_updated = video_description_line1.replace("#EP", str(ep))
                video_description_line2_updated = "\n" + video_description_line2 if video_description_line2 else ""
                video_description = video_description_line1_updated + video_description_line2_updated

                print(f"Uploading episode {ep} to Facebook...")
                print(f"Title: {video_title}")
                print(f"Description: {video_description}")
                print(f"File path: {file_path}")

                upload_successful, response, video_id = upload_ep_2_fb(file_path, video_title, video_description, page_id, access_token)

                if upload_successful:
                    print(f"Episode {ep} uploaded successfully to Facebook.")
                else:
                    print(f"Failed to upload episode {ep} to Facebook.")
                    break
            else:
                print(f"Failed to download episode {ep}.")

    except Exception as e:
        print(f"An error occurred: {e}")

#################################################
################## End Program ##################
#################################################

# Example usage
if save_method == "google sheet":
    download_and_upload_episodes_with_save(start_ep, end_ep, url, page_id, access_token, folder_id)
elif save_method == "without save":
    download_and_upload_episodes_without_save(start_ep, end_ep, url, page_id, access_token)
